In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

from estadistica.calculos import generar_tabla_porcentajes, calcular_porcentajes_por_est_socio_m2
from estadistica.calculos import generar_tabla

In [2]:
f = '../data/001_raw/ENCEVI_2018/encevi.csv'
columnas = ['folio','tipo_viv','total_niv','super_cons',
            'super_ter','tot_cuart','electri','local_com',
            # 'cons_med1','cons_med2','inicia1','mes_inicia1',
            # 'inica2','mes_inicia2',
           ]
encevi = pd.read_csv(f,usecols=columnas)
columnas = encevi.columns
encevi.set_index('folio', inplace=True)

f = '../data/001_raw/ENCEVI_2018/vivienda.csv'
columnas = ['folio','factor_sem','est_socio','region']
vivienda = pd.read_csv(f,usecols=columnas)
vivienda.set_index('folio',inplace=True)

vivienda = vivienda.join(encevi,how='inner')
vivienda.factor_sem.sum()

# Supongamos que tu dataframe se llama vivienda y la columna con el código es 'super_cons'
mapeo_m2 = {
    1: '30',
    2: '31_55',
    3: '56_75',
    4: '76_100',
    5: '101_150',
    6: '151_200',
    7: '200_+',
    9: 'No sabe'
}
mapeo_estrato = {
    1: 'Bajo',
    2: 'Medio bajo',
    3: 'Medio alto',
    4: 'Alto',
}

mapeo_clima = {
    1:'Cálida extrema',
    2:"Templada",
    3:"Tropical"
}

# Crear la nueva columna "m2" aplicando el diccionario de mapeo
vivienda['m2'] = vivienda['super_cons'].map(mapeo_m2)
vivienda['est_socio'] = vivienda['est_socio'].map(mapeo_estrato)
vivienda['clima'] = vivienda['region'].map(mapeo_clima)

original = vivienda.copy()
original.factor_sem.sum()

np.int64(33162148)

In [3]:
f = '../data/001_raw/ENCEVI_Hernandez/encevi_result_db_041722.csv'
columnas = ['folioviv','bill_kwh_estimation','bill_days']
energia = pd.read_csv(f,usecols=columnas,index_col=0)

vivienda = energia.join(vivienda,how='inner')
vivienda.index.names = ['folio']
print(vivienda.factor_sem.sum())

22337082


In [4]:
calcular_porcentajes_por_est_socio_m2(original)

m2,30,31_55,56_75,76_100,101_150,151_200,200_+,No sabe,Total
est_socio,,,,,,,,,
Bajo,2.6,3.8,2.7,2.4,1.4,0.6,0.3,3.4,17.2
Medio bajo,4.5,8.0,8.0,8.1,6.0,3.0,1.8,11.7,51.1
Medio alto,0.7,1.7,3.8,3.9,3.2,1.7,1.0,5.7,21.7
Alto,0.1,0.6,1.5,1.8,2.0,1.1,1.0,2.1,10.2


In [5]:
calcular_porcentajes_por_est_socio_m2(vivienda)

m2,30,31_55,56_75,76_100,101_150,151_200,200_+,No sabe,Total
est_socio,,,,,,,,,
Bajo,1.6,3.5,2.8,2.5,1.5,0.5,0.4,2.6,15.4
Medio bajo,3.0,7.4,8.2,8.6,6.5,3.1,2.0,10.0,48.8
Medio alto,0.6,1.9,4.6,4.5,3.7,1.9,1.2,5.8,24.2
Alto,0.1,0.6,1.8,2.2,2.2,1.3,1.1,2.2,11.5


In [6]:
calcular_porcentajes_por_est_socio_m2(original) - calcular_porcentajes_por_est_socio_m2(vivienda)

m2,30,31_55,56_75,76_100,101_150,151_200,200_+,No sabe,Total
est_socio,,,,,,,,,
Bajo,1.0,0.3,-0.1,-0.1,-0.1,0.1,-0.1,0.8,1.8
Medio bajo,1.5,0.6,-0.2,-0.5,-0.5,-0.1,-0.2,1.7,2.3
Medio alto,0.1,-0.2,-0.8,-0.6,-0.5,-0.2,-0.2,-0.1,-2.5
Alto,0.0,0.0,-0.3,-0.4,-0.2,-0.2,-0.1,-0.1,-1.3


In [7]:
generar_tabla_porcentajes(original)

In [8]:
generar_tabla_porcentajes(vivienda)

In [9]:
(generar_tabla_porcentajes(original,devolver_styler=False) - generar_tabla_porcentajes(vivienda,devolver_styler=False)).style.format("{:.1f}").background_gradient(axis=0).bar(subset=['Total'], color='#FFA07A')

In [10]:
# vivienda.to_parquet('../data/002_intermediate/vivienda_energia_m2_todo.parquet')

In [11]:
tabla_orig   = generar_tabla(original, devolver_styler=False)
tabla_limpia = generar_tabla(vivienda,   devolver_styler=False)

# diferencia absoluta (limpia – original)
diferencia   = tabla_limpia - tabla_orig
diferencia

est_socio                 Bajo  Medio bajo  Medio alto   Alto    Total
clima          m2                                                     
Cálida extrema 30       -40416     -103814      -27494  -1083  -172807
               31_55    -36784     -216354      -60196 -20659  -333993
               56_75    -19857     -249359      -84538 -23509  -377263
               76_100   -28547     -170485     -103918 -61867  -364817
               101_150  -11929     -138677     -104319 -58756  -313681
               151_200   -8366      -58125      -56008 -36507  -159006
               200_+     -3522      -41679      -29753 -47934  -122888
               No sabe  -39428     -304702     -149122 -58135  -551387
Templada       30      -183051     -575067      -44028 -10599  -812745
               31_55   -140069     -605625      -65598 -26209  -837501
               56_75    -98515     -452488     -145183 -48616  -744802
               76_100   -76865     -443606     -154976 -33272  -708719
               101_150  -29955     -304161     -116876 -75571  -526563
               151_200  -13041     -206880      -76370 -49064  -345355
               200_+    -12306      -64603      -25438 -36389  -138736
               No sabe -129449     -850401     -363520 -92586 -1435956
Tropical       30      -263691     -157815      -10812  -1269  -433587
               31_55   -316201     -190534       -7494  -3582  -517811
               56_75   -140957     -115559      -11898  -4585  -272999
               76_100  -136644     -136010      -12714  -6405  -291773
               101_150  -68757      -81148      -12229 -10053  -172187
               151_200  -45030      -38102       -8431  -6841   -98404
               200_+    -18908      -47883       -6139  -4837   -77767
               No sabe -375934     -515038      -78376 -44971 -1014319

In [12]:
vivienda.stack().values

array([62, 113.8023013, 'Medio alto', ..., '2', 'No sabe', 'Templada'],
      shape=(288022,), dtype=object)

In [13]:

# Tablas de porcentaje que obtuviste con generar_tabla_porcentajes
tabla_orig   = generar_tabla_porcentajes(original, devolver_styler=False)
tabla_limpia = generar_tabla_porcentajes(vivienda,   devolver_styler=False)

N_orig   = original['factor_sem'].sum()
N_limpia = vivienda  ['factor_sem'].sum()

frec_orig   = (tabla_orig   * N_orig).stack().values
frec_limpia = (tabla_limpia * N_limpia).stack().values

cont = np.vstack([frec_orig, frec_limpia])

chi2, p, dof, _ = chi2_contingency(cont, correction=False)

r = cont.shape[0]          # 2 filas (origen vs limpio)
c = cont.shape[1]          # k celdas
k = min(r, c)              # **debe ser 2** (no 120) si mantienes la tabla 2×k

V = np.sqrt(chi2 / (cont.sum() * (k - 1)))
print(f"χ² = {chi2:,.0f},  g.l. = {dof},  p = {p:.4f},  Cramér V = {V:.3f}")


χ² = 63,091,820,  g.l. = 119,  p = 0.0000,  Cramér V = 0.075


In [14]:
vivienda.bill_kwh_estimation.mean()

np.float64(174.94333686821383)

In [15]:
vivienda.factor_sem.sum()

np.int64(22337082)